In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.ensemble import HistGradientBoostingRegressor
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

# Initialize lists to collect model performance
model_names = []
model_maes = []
model_mabs = []

# Read, pre-process and visualize data

In [2]:
data_set = pd.read_csv('MultipleSources-2019-2025.csv')
data_set.head()

,Date,SP500,Futures,Nikkei,FTSE,DAX
0,2025-04-02,"5,670.97","5,512.00","35,725.87","8,608.48","22,390.84"
1,2025-04-01,"5,633.07","5,674.50","35,624.48","8,634.80","22,539.98"
2,2025-03-31,"5,611.85","5,653.25","35,617.56","8,582.81","22,163.49"
3,2025-03-28,"5,580.94","5,623.00","37,120.33","8,658.85","22,461.52"
4,2025-03-27,"5,693.31","5,739.25","37,799.97","8,666.12","22,678.74"


In [3]:
len(data_set)
data_set.isna().sum()
data_set.dtypes

1509

Date        0
SP500       0
Futures     0
Nikkei     98
FTSE       30
DAX        24
dtype: int64

Date        object
SP500      float64
Futures    float64
Nikkei     float64
FTSE       float64
DAX        float64
dtype: object

In [4]:
data_set.columns

# Step 1: Convert to datetime
data_set['Date'] = pd.to_datetime(data_set['Date'])

# ✅ Step 2: Extract day of week
data_set['day_of_week'] = data_set['Date'].dt.dayofweek  # 0=Mon, ..., 6=Sun

# Step 3 (optional): If you still want UNIX timestamp
data_set['Date'] = data_set['Date'].astype(np.int64) // 10**9

data_set.tail()

Index(['Date', 'SP500', 'Futures', 'Nikkei', 'FTSE', 'DAX'], dtype='object')

,Date,SP500,Futures,Nikkei,FTSE,DAX,day_of_week
1504,1554768000,"2,878.20","2,882.50","21,802.59","7,425.57","11,850.57",1
1505,1554681600,"2,895.77","2,898.25","21,761.65","7,451.89","11,963.40",0
1506,1554422400,"2,892.74","2,896.00","21,807.50","7,446.87","12,009.75",4
1507,1554336000,"2,879.39","2,882.75","21,724.95","7,401.94","11,988.01",3
1508,1554249600,"2,873.40","2,879.75","21,713.21","7,418.28","11,954.40",2


In [5]:
# Reshape to 2D as required by OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
day_of_week_encoded = encoder.fit_transform(data_set[['day_of_week']])

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:
# Get feature names like ['day_of_week_0', 'day_of_week_1', ...]
encoded_cols = encoder.get_feature_names_out(['day_of_week'])
encoded_df = pd.DataFrame(day_of_week_encoded, columns=encoded_cols, index=data_set.index)
data_set = pd.concat([data_set.drop(columns=['day_of_week']), encoded_df], axis=1)

In [ ]:
def add_lag_features(df, columns, lags):
    """
    Adds lag features for given columns and lag days.

    Parameters:
    df (pd.DataFrame): Original dataset
    columns (list): Columns for which to create lag features
    lags (list): List of lag values (e.g., [1, 2, 3])

    Returns:
    pd.DataFrame: Dataset with new lag features
    """
    for col in columns:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    return df


lag_columns = ['SP500', 'DAX', 'FTSE', 'Nikkei']
lag_days = [1, 2, 3] 

data_set = add_lag_features(data_set, lag_columns, lag_days)
data_set = data_set.dropna()  # Drop rows with NaNs from lags
data_set.head()


data_set = data_set.sort_values(by='Date').reset_index(drop=True)



In [ ]:
data_set['SP500_next'] = data_set['SP500'].shift(-1)
data_set = data_set.dropna()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_set.drop(columns = ['SP500_next']), data_set['SP500_next'], test_size=0.2,shuffle = False)
## Removing the Date feature resulted in higher Mean Absolute Error (MAE) across all models
X_train
X_test
y_train
y_test

# Decison tree


In [ ]:
clf = DecisionTreeRegressor(random_state=50)

clf = clf.fit(X_train, y_train) 

In [ ]:
X_train.columns
clf.feature_importances_

In [ ]:
test_output = pd.DataFrame(clf.predict(X_test), index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Decision Tree")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

## Bagging Regressor ## 

In [ ]:
regr = BaggingRegressor(random_state=50, n_estimators = 200, max_samples = 800)

regr = regr.fit(X_train, y_train) 

In [ ]:
test_output = pd.DataFrame(regr.predict(X_test), index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Bagging Regressor")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

## Random Forest Regressor 

In [ ]:
rf = RandomForestRegressor(random_state=50, min_samples_leaf = 3, max_features = "sqrt")

rf = rf.fit(X_train, y_train) 


In [ ]:
X_train.columns
rf.feature_importances_

In [ ]:
test_output = pd.DataFrame(rf.predict(X_test), index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Random Forest Regressor")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

## Gradeint Boosting Regressor

In [ ]:
gb = GradientBoostingRegressor(random_state=50, min_samples_leaf = 2, max_depth = 4)

gb = gb.fit(X_train, y_train) 


In [ ]:
X_train.columns
gb.feature_importances_

In [ ]:
test_output = pd.DataFrame(gb.predict(X_test), index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Gradeint Boosting Regressor")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

## XGBoost Regressor 

In [ ]:
# XGBoost comes with its own class for storing datasets called DMatrix. 
# It is a highly optimized class for memory and speed. 
# That's why converting datasets into this format is a requirement for the native XGBoost API:


# Create regression matrices

dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)

dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [ ]:
params = {"objective": "reg:squarederror", "tree_method": "exact", "max_depth" : 4, "learning_rate" : 0.1} # use "tree_method" : "hist" if you need speed

In [ ]:
n = 100

model = xgb.train(

   params=params,

   dtrain=dtrain_reg,

   num_boost_round=n,

)

In [ ]:
from sklearn.metrics import mean_squared_error
preds = model.predict(dtest_reg)


In [ ]:
test_output = pd.DataFrame(preds, index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("XGBoost Regressor")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

## Hybrid Model 

In [ ]:
model = LinearRegression(fit_intercept = True)
model.fit(X_train, y_train) 

# The following gives the R-square score
model.score(X_train, y_train) 

In [ ]:
training_residuals = y_train - model.predict(X_train)

In [ ]:
rf = RandomForestRegressor(random_state=50, min_samples_leaf = 3, max_features = "sqrt")

rf = rf.fit(X_train, training_residuals) 

In [ ]:
pred_residuals = rf.predict(X_test)
y_pred = pred_residuals + model.predict(X_test)

In [ ]:
test_output = pd.DataFrame(y_pred, index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Hybrid Model")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

# Hybrid (XGBoost)

In [ ]:
# XGBoost on residuals
dtrain_resid = xgb.DMatrix(X_train, label=training_residuals, enable_categorical=True)
dtest_resid = xgb.DMatrix(X_test, enable_categorical=True)

In [ ]:
params = {"objective": "reg:squarederror", "tree_method": "exact", "max_depth" : 4, "learning_rate" : 0.1} # use "tree_method" : "hist" if you need speed
num_round = 100

In [ ]:
residual_model = xgb.train(params, dtrain_resid, num_boost_round=num_round)

# Predict residuals on test set using XGBoost
pred_residuals = residual_model.predict(dtest_resid)

# Final hybrid prediction = linear prediction + xgboost residual prediction
yy_pred = model.predict(X_test) + pred_residuals

In [ ]:
test_output = pd.DataFrame(yy_pred, index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Hybrid (XGBoost)")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

## Plotting a Graph for Comparing Results


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.DataFrame({'Model': model_names, 'MAE': model_maes})

plt.figure(figsize=(10, 6))
bars = plt.bar(df['Model'], df['MAE'], color='cornflowerblue', edgecolor='black')
plt.title('Comparison of Mean Absolute Error (MAE) Across Models', fontsize=14)
plt.xlabel('Models')
plt.ylabel('MAE (lower is better)')
plt.xticks(rotation=45)

# Annotate bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.5, f'{yval:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show();


# Plotting a Graph using MAPE

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create DataFrame with model names and MAPE values
df = pd.DataFrame({'Model': model_names, 'MAPE': model_mabs})

# Create the plot
plt.figure(figsize=(10, 6))
bars = plt.bar(df['Model'], df['MAPE'], color='cornflowerblue', edgecolor='black')

# Add title and axis labels
plt.title('Comparison of MAPE Across Models', fontsize=16)
plt.xlabel('Machine Learning Models', fontsize=12)
plt.ylabel('MAPE (Lower is better)', fontsize=12)
plt.xticks(rotation=45)

# Annotate each bar with its height (as percentage)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.0005, f"{yval:.2%}", 
             ha='center', va='bottom', fontsize=10)

# Improve layout
plt.tight_layout()
plt.show();
